In [1]:
from perceval.backends.core.github import GitHub
from datetime import date
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import os
import sqlalchemy as s
import augur
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize 
import nltk.data
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import sentiment
from nltk import word_tokenize

In [2]:
if("notebooks" not in os.getcwd()):
    os.chdir("notebooks")
augurApp = augur.Application('../augur.config.json')
connect = augurApp.ghtorrentplus()
DB_STR = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format(
            'derek', 'picklehead5000', 'ghdata.sociallycompute.io',\
            '3306', 'chaoss'
        )
db = s.create_engine(DB_STR)

In [3]:
table_names = s.inspect(db).get_table_names()
print(table_names)

['github_issues', 'issue_response_time', 'mail_lists', 'mail_lists_sentiment_scores', 'mailing_list_jobs']


In [4]:
if("github_issues" in table_names):
    lists_createdSQL = s.sql.text("""SELECT repo FROM github_issues""")
    df1 = pd.read_sql(lists_createdSQL, db)
    #print(df1)
    val = True

In [5]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sid = SentimentIntensityAnalyzer()
col = 'score','sentiment'

In [6]:
SQL = """SELECT * FROM github_issues"""
df_users =  pd.read_sql(SQL, db)
#print(df_users)

In [7]:
grouped = df_users.groupby('subject').groups
num = 0
col = 'score','sentiment'
columns1 = 'augurmsgID',"backend_name",'repo_link',"owner","repo","subject",\
          "status","category","issue_number","timestamp",\
          "issue_id","user","body"
df3 = pd.DataFrame(columns=col)
df_list = pd.DataFrame(columns = columns1)
df_list = df_list.append(df_users)
for group in grouped:
    #print(group)
    issue_num = (df_users.loc[df_users['subject'] == group]['issue_number']).values
    messages = (df_users.loc[df_users['subject'] == group]['body']).tolist()
    id1 = (df_users.loc[df_users['subject'] == group]['body']).tolist()
    #print("Issue number: ",issue_num[0])
    #break
    for i in messages:
        #print(i)
        sentences = tokenizer.tokenize(i)
        compound = 0
        part = 0
        sentiment = "Positive"
        for sentence in sentences:
            scores = sid.polarity_scores(sentence)
            compound+=scores['compound']
            part+=1
            #print(sentence,scores)
            #if(scores['compound'] < 0.1):
            #    print(sentence,scores['compound'])
        avg = compound/part
        avg = round(avg,3)
        if(avg == 0):
            sentiment = "Neutral"
        elif(avg < 0):
            sentiment = "Negative"
        li = [ [avg,sentiment]]
        df_temp = pd.DataFrame(li,columns=col)
        df3 = df3.append(df_temp)
        #if(avg < -0.5):
            #print(i,avg)
            #print("\n\n\n\n\n\n")
    #if(num == 40):
    #    break
    num+=1
    #print("\n\n\n")
print(num)

126


In [8]:
#print(df3)
print(df_list)
df3 = df3.reset_index(drop=True)
print(df3.head())
df_list = df_list.reset_index(drop=True)
combine = (df_list.join(df3))

     augurmsgID backend_name                      repo_link   owner repo  \
0             1       GitHub  https://github.com/nodejs/CTC  nodejs  CTC   
1             2       GitHub  https://github.com/nodejs/CTC  nodejs  CTC   
2             3       GitHub  https://github.com/nodejs/CTC  nodejs  CTC   
3             4       GitHub  https://github.com/nodejs/CTC  nodejs  CTC   
4             5       GitHub  https://github.com/nodejs/CTC  nodejs  CTC   
5             6       GitHub  https://github.com/nodejs/CTC  nodejs  CTC   
6             7       GitHub  https://github.com/nodejs/CTC  nodejs  CTC   
7             8       GitHub  https://github.com/nodejs/CTC  nodejs  CTC   
8             9       GitHub  https://github.com/nodejs/CTC  nodejs  CTC   
9            10       GitHub  https://github.com/nodejs/CTC  nodejs  CTC   
10           11       GitHub  https://github.com/nodejs/CTC  nodejs  CTC   
11           12       GitHub  https://github.com/nodejs/CTC  nodejs  CTC   
12          

In [9]:
print(combine.head())

  augurmsgID backend_name                      repo_link   owner repo  \
0          1       GitHub  https://github.com/nodejs/CTC  nodejs  CTC   
1          2       GitHub  https://github.com/nodejs/CTC  nodejs  CTC   
2          3       GitHub  https://github.com/nodejs/CTC  nodejs  CTC   
3          4       GitHub  https://github.com/nodejs/CTC  nodejs  CTC   
4          5       GitHub  https://github.com/nodejs/CTC  nodejs  CTC   

                 subject  status category issue_number   timestamp   issue_id  \
0  Purpose of nodejs/CTC  closed    issue            1  1532348853  159398443   
1  Purpose of nodejs/CTC  closed    issue            1   159398443          1   
2  Purpose of nodejs/CTC  closed    issue            1   159398443          1   
3  Purpose of nodejs/CTC  closed    issue            1   159398443          1   
4  Purpose of nodejs/CTC  closed    issue            1   159398443          1   

          user                                               body  score  

In [10]:
combine.to_sql(name='github_issues_sentiment_scores',\
               con=db,if_exists='replace',index=False)